In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
tf.__version__

In [ ]:
keras.__version__

In [ ]:
model = keras.Sequential()
# 第一层是输入层，无需制定输入的shape，会自动匹配，也不需要activation？
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation="softmax"))

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [ ]:
data = np.random.random((1000,32))
label = np.random.random((1000,10))

In [ ]:
history = model.fit(data, label, epochs=10, batch_size=32)